# Anomaly Detection Tool
In this notebook, we outline a process for running a simple statistical analysis for anomalies on a set of geospatial data. Below, you can find some helpful frameworks for pulling a batch of data for multiple regions, transforming that data into a more statistically significant format, identifying key properties of that data, and then identifying if recent values are anomalous according to two sigma significance. This code is extendable to include different tests for significance, different data sets and different use cases.

## Set up the coding environment

In [18]:

from api.client.gro_client import GroClient
from datetime import datetime, date, timedelta
import pandas as pd
import os
import re


In [3]:

GROAPI_TOKEN = os.environ['GROAPI_TOKEN']
API_HOST = 'api.gro-intelligence.com'
gclient = GroClient(API_HOST, GROAPI_TOKEN)


In [4]:

#data_name = "Salta Province in Brazil Rainfall Anomalies"
data_name = "Salta Province in Argentina Temperature Anomalies"
salta_region_id = 10152
districts_list = gclient.lookup('regions', salta_region_id)['contains']
print(len(districts_list), "district ids:" )
print(districts_list)


24 district ids:
[102446, 102454, 102459, 102463, 102453, 102449, 102458, 102451, 102448, 102457, 102447, 102465, 102467, 102462, 102464, 102455, 102468, 102461, 102450, 102452, 102460, 102456, 102466, 100022792]


## Now we need a function to help us the best data series for multiple locations

We want to choose the sereis with the best rank so we use this handy method to retrieve the best


In [5]:

def best_series_picker(selection):
    '''
    chooses the series with the greatest density and history of datapoints
    '''
    temp_series = (next(gclient.rank_series_by_source(selection)))
    temp_series = gclient.get_data_series(**temp_series)
    temp_series = (temp_series[0])
    return temp_series


You can get the ids necessary to pull a data series at gro-intelligence.com by searching for the terms you need. in this case we are going to look at Land temperature (daytime, modeled) from the MOD11 Satellite housed on gro.

In [6]:

def get_land_temp(regions_list):
    '''
    # Land temperature (daytime, modeled) - Temperature - (MOD11 LST)
    '''
    selections = []
    for region in regions_list:
        '''
        ids from gro-intelligence.com
        '''
        selections.append(best_series_picker([{'metric_id': 2540047, 'item_id': 3457, 'region_id': region}])) #you can get these id's from our web app at gro-intelligence.com
    
    output = []
    for set_of_ids in selections:
        output.append(gclient.get_data_points(**set_of_ids))
    
    return output



## Time to pull data from the Gro API and examine what we have. (This may take a few seconds)

In [7]:

all_data_array = get_land_temp(districts_list)
# We will use all_data_array later!


Check on that data by printing one data point! (if you try to print the whole thing the notebook will crash)


In [8]:

first_location_data = all_data_array[0]
first_data_point = first_location_data[-1]
print(first_data_point)


{'start_date': '2019-12-10T00:00:00.000Z', 'end_date': '2019-12-10T00:00:00.000Z', 'value': 37.0214822237871, 'input_unit_id': 36, 'input_unit_scale': 1, 'unit_id': 36, 'metric_id': 2540047, 'item_id': 3457, 'region_id': 102446, 'frequency_id': 1}


Lets look at what we are working with:


In [9]:

most_recent_value = (first_location_data[-1]['value'])
item = gclient.lookup('items',first_location_data[0]['item_id'])['name']
unit = gclient.lookup('units',first_location_data[0]['unit_id'])['name']
region = gclient.lookup('regions',first_location_data[0]['region_id'])['name']
metric_id = first_location_data[0]['metric_id']
metric_name = gclient.lookup('metrics',metric_id)['name']
time_stamp = " on " + first_location_data[-1]['end_date'][:10] + " "
frequency = gclient.lookup('frequencies',first_location_data[0]['frequency_id'])['name']
years = datetime.strptime(first_location_data[-1]['end_date'][:10], '%Y-%m-%d').year - datetime.strptime(first_location_data[0]['end_date'][:10], '%Y-%m-%d').year

print("most recently reported value", most_recent_value)
print("item name:", item)
print("unit name:", unit)
print("name of region:", region)
print("name of metric:", metric_name)
print("last reported data point:", time_stamp)
print("frequency reported:", frequency)
print("time period (years):", years)


most recently reported value 37.0214822237871
item name: Land temperature (daytime, modeled)
unit name: Celsius
name of region: Anta
name of metric: Temperature
last reported data point:  on 2019-12-10 
frequency reported: daily
time period (years): 19


Next we will want to give our users friendly wording when they read their anomaly alerts
this is optional but for many readers "Water Loss" makes more sense than "Evapotranspiration"
the variable data is defined as one series from the all_data_array

In [10]:

def check_if_geospatial(data):
    metric_id = data[0]['metric_id']
    if(metric_id==2100031 or metric_id ==15531082 or metric_id == 430029 or metric_id ==2010042 or metric_id == 2540047):
        is_geospatial = True
    else:
        is_geospatial =  False
    if(metric_id==2100031):# Trmm
        metric_name = 'Rainfall'
    elif(metric_id==15531082):# Soil Moisture
        metric_name = 'Soil Moisture'
    elif(metric_id==430029):# NDVI
        metric_name = 'Vegetation Health'
    elif(metric_id==2010042):# ETA
        metric_name = 'Water Loss'
    elif(metric_id==2540047):# Temp
        metric_name = 'Temperature'
    else:
        metric_name = ''

    return is_geospatial, metric_name


In [11]:

# Lets try it out on our first data location
is_geospatial, new_metric_name = check_if_geospatial(first_location_data)
print("geospatial data:", is_geospatial)
print("type:", new_metric_name)


geospatial data: True
type: Temperature


## Transform data for more significant signals

For some data series we dont want to look at values as frequently as they are reported, instead we want to look at 7-day cumulative or 7-day average values. Here we will modify those specified series and Temperature is one of them! 

In [29]:

def transform_by_data_type(df, current_value, frequency):
    '''
    change frequency for certain weather items
    '''
    metric_id = df.loc[0,'metric_id']
    if(metric_id == 2540047 or metric_id == 15531082): # 2540047 is Land temp, 15531082 is soil moisture (MEAN)
        df = df.fillna(df.mean())
        df['value'] = df['value'].rolling(7).mean()
        frequency = "7-day average"
    elif( metric_id == 2100031):
        df = df.fillna(0)
        df['value'] = df['value'].rolling(7).sum() # 2100031 is rainfall (SUM)
        frequency = "7-day cumulative"
    else:
        return df, current_value, frequency
    df = df.iloc[::7, :]
    df.index = pd.RangeIndex(len(df.index))
    current_value = df.iloc[-1]['value']
    return df, current_value, frequency


# Get percent change from average
df = pd.DataFrame(first_location_data)

df, current_value, frequency = transform_by_data_type(df, most_recent_value, frequency)

print("updated frequency:", frequency)
print("most recent 7-day average value:", current_value)


updated frequency: 7-day average
most recent 7-day average value: 31.085297007615953


## Create a significance / anomaly test

Now we need to find the lower and upper bounds for our testing for significance, we have chosen to consider two standard deviations from the mean. After that we will use the bounds to test for significant deviations from the mean and record highs and lows. This test could be replaced with any other test for significance.



In [13]:

def find_lower_and_upper(df,standard_devs):
    '''
    Standard Deviation Calculations for Alerts:
    Using standard deviations we cand find out if recent numbers
    reported to the user are abnormal or to be expected
    Returns upper and lower bounds for a series and a given number of standard deviations
    '''
    mean = df.mean()
    std = df.std()
    lower_bound = (mean - (standard_devs * std))
    upper_bound = (mean + (standard_devs * std))
    return lower_bound, upper_bound


df_lower_upper = df['value']
standard_deviations = 2
lower_bound, upper_bound = find_lower_and_upper(df_lower_upper,standard_deviations)

print("lower bound:", lower_bound)
print("upper bound:", upper_bound)


lower bound: 16.156849838891347
upper bound: 35.78526612313051


In [22]:

def selected_tests(df,lower_bound,upper_bound,current_value):
    is_significant = False
    statement = ""
    if(current_value == df["value"].max()):
        is_max = True
        statement = "Record High in "
    elif(current_value == df["value"].min()):
        is_max = True
        statement = "Record Low in "
    elif(current_value<=lower_bound):
        is_significant = True
        statement = "Significant low in "
    elif(current_value>=upper_bound):
        is_significant = True
        statement = "Significant high in "
    else:
        statement = "Not significant for period: "
    return is_significant, statement


is_significant, statement = selected_tests(df,lower_bound,upper_bound,current_value)

statement += str(years) + " years "
current_value = ((round(current_value,2)))

print("test result:", statement)


test result: Not significant for period: 19 years 


We also want to contextualize this number by saying exactly what the deviation from the mean looks like

In [15]:

def get_percent_change_statement(number, avg, metric_id):
    if(metric_id == 2010042): #ETA
        percent_val = number/100
        deviation = number - 100
    elif(metric_id == 430029): #NDVI
        percent_val = abs(number)
        deviation = number
    else:
        deviation = (number) - (avg)
        if(avg!=0):
            percent_val = (abs(deviation) / abs(avg))
        else:
            return ''
    # Format string response
    if(deviation>=0):
        statement = str("{0:.0%}".format(percent_val)) + " above average"
    else:
        statement = str("{0:.0%}".format(percent_val)) + " below average"

    return statement

mean = df["value"].mean()
percent_change = get_percent_change_statement(current_value, mean, metric_id)

print("percent deviation:", percent_change)


percent deviation: 20% above average


## Putting it all together

After walking through the process we want to add some methods so we can integrate this into our workflow and use it for many data series at a time.

In [32]:

def current_is_significant(data):
    '''
    Returns bool True if the latest value is the global max
    '''
    current_value = (data[-1]['value'])
    frequency = gclient.lookup('frequencies',data[0]['frequency_id'])['name']

    df = pd.DataFrame(data)
    df, current_value, frequency = transform_by_data_type(df, current_value, frequency)
    current_value = df.iloc[-1]['value']

    df_lower_upper = df['value']
    standard_deviations = 2
    lower_bound, upper_bound = find_lower_and_upper(df_lower_upper,standard_deviations)
    
    is_significant, statement = selected_tests(df,lower_bound,upper_bound,current_value)
    
    # Get frequency
    years = datetime.strptime(data[-1]['end_date'][:10], '%Y-%m-%d').year - datetime.strptime(data[0]['end_date'][:10], '%Y-%m-%d').year
    statement += str(years) + " years "
    return is_significant, statement, ((round(current_value,2)))


def return_alerts(data):
    alert = ""
    '''
    alert if current data is significant in gro series
    make sure there was data in the original request before running the rest
    '''
    #try:
    # Make sure there are more than 3 data points
    data[3]
    if(data[-1]["value"]==data[-6]["value"]):
        return ""
    # Make sure the most recent reporting date is within the last 30 days
    current_date = datetime.now()
    try:
        most_recent = data[-1]['reporting_date']
    except:
        most_recent = data[-1]['end_date']
    data_date = datetime.strptime(most_recent[:10], '%Y-%m-%d')
    delta = current_date-data_date
    days = int(str(delta).split()[0])
    if(days>=30):
        return ""
    
    # alert if current is max
    bin_val1,statement1,val1 = current_is_significant(data)
    if(bin_val1):
        alert += " - " + statement1
    
    ''' You can add tests for seasonal significance, tests for alternate distributions etc.
    bin_val2,statement2,val2 = YOUR_TEST_HERE(data)  
    if(bin_val2):
        alert += " - " + statement2
    '''
    return alert


def string_formatting(data, number):
    try:
        # Make sure there was data returned 
        most_recent_value = (data[number]['value'])
        
        # Provide metadata for the reader from gro api client and format into human readable string
        item = gclient.lookup('items',data[0]['item_id'])['name']
        unit = gclient.lookup('units',data[0]['unit_id'])['name']
        region = gclient.lookup('regions',data[0]['region_id'])['name']
        metric_id = data[0]['metric_id']
        metric_name = gclient.lookup('metrics',metric_id)['name']
        time_stamp = " on " + data[number]['end_date'][:10] + " "
        frequency = gclient.lookup('frequencies',data[0]['frequency_id'])['name']
        
    except (TypeError, KeyError):
        print("type error or key error in string_formatting")
        return "", ''

    # Get percent change from average
    df = pd.DataFrame(data)
    df, current_value, frequency = transform_by_data_type(df, most_recent_value, frequency)
    mean = df["value"].mean()
    percent_change = get_percent_change_statement(most_recent_value, mean, metric_id)

    # Check if this is a geospatial data series
    is_geospatial, new_metric_name = check_if_geospatial(data)

    # If this IS a geospatial metric
    if(is_geospatial):
        #create string
        significance_headline = region+ " - " + new_metric_name + ": " + percent_change

    # If this is NOT a geospatial metric
    else:
        # Make sure it exists
        if(most_recent_value):x
            most_recent_value = round(most_recent_value,2)
        else:
            most_recent_value = 0

        #Adds commas and gets rid of decimals
        if(most_recent_value>100):
            most_recent_value = (format(int(most_recent_value), ',d'))
        else:
            most_recent_value = str(most_recent_value)
        # create string
        item_metric = item + " - " + metric_name
        significance_headline = region+ " - " + item_metric + ": " + most_recent_value + " " + unit + ", " + percent_change
    # removes words within parentheses for easier reading
    significance_headline = (re.sub(r" ?\([^)]+\)", "", significance_headline))
    return significance_headline


def work_to_distribute(data):
    '''
    Method run by workers, this will run each of the tests in return_alerts
    '''
    string_add = ""
    data_line_2 = return_alerts(data)
    if(data_line_2!=""):
        data_line_1 = string_formatting(data,-1)
        string_add += (data_line_1 + data_line_2 + "\n")
    return string_add


def recent_data_to_string(array, data_name):
    '''
    Takes array of data series and a category name and returns any alerts string formatted under a header
    '''
    string = ""
    string_add = ""
    string_list = []
    if(array != None):
        try:
            for ele in array:
                string_list.append(work_to_distribute(ele))
        except IndexError:
            print("IndexError in recent_data_to_string for " + data_name)
            return string_add
            
        for x in string_list:
            string_add += x
        if(string_add!=""):
            string = data_name
        else:
            string = ""

    data_statement = string + " \n" + string_add

    return data_statement


argentina_string = recent_data_to_string(all_data_array, data_name)
print(argentina_string)



## Introducing new data

Now that we have our alerts from Argentina about Land Temperature (If the cell above prints nothing, that means there were no alerts for significant temperature in Argentina), we will try brazil NDVI (Normalized Difference Vegetation Index) anomalies as well. Our data source for NDVI allows us to estimate the amound of chlorophyll in plants in a given region.

In [33]:

def get_ndvi_10_year_fixed(regions_list):
    '''
    # NDVI difference from 10-yr mean (2001-2010) - Vegetation Anomalies
    '''
    selections = []
    for region in regions_list:
        '''
        ids from gro-intelligence.com
        '''
        #you can get these id's from our web app at gro-intelligence.com
        selections.append(best_series_picker([{'metric_id': 430029, 'item_id': 409, 'region_id': region}]))    
    
    output = []
    for set_of_ids in selections:
        output.append(gclient.get_data_points(**set_of_ids))
    
    return output

# Tocantins Province in Brazil
tocantins_province_id = 10428
districts_list_brazil = gclient.lookup('regions', tocantins_province_id)['contains']
print(districts_list_brazil)


[110117, 110031, 110130, 110097, 110120, 110059, 110124, 110047, 110041, 110118, 110055, 110061, 110084, 110082, 110096, 110077, 110109, 110100, 109997, 110045, 110021, 110074, 109998, 110022, 110024, 110086, 110001, 109995, 110048, 110064, 110004, 110098, 110016, 110037, 110010, 110078, 110107, 109999, 110051, 110017, 110091, 110116, 110023, 110131, 110089, 110065, 110088, 110028, 110002, 110090, 110115, 110092, 110018, 110104, 110052, 110056, 110011, 110106, 110085, 110083, 110035, 110122, 110087, 110121, 110006, 109996, 110068, 110014, 110034, 110057, 110066, 110102, 110042, 110071, 110039, 110053, 110040, 110007, 110094, 110049, 110030, 110073, 110029, 110129, 110076, 110105, 110075, 110067, 110062, 110113, 110114, 110000, 110020, 110046, 110050, 110063, 110101, 110110, 110128, 110036, 110093, 110069, 110012, 110003, 110103, 110080, 110127, 110099, 110060, 110015, 110008, 110033, 110070, 110027, 110095, 110079, 110112, 110019, 110013, 110005, 110125, 110111, 110108, 110072, 110119,

## Pull Tocantins province NDVI and analyze for significance (this may take a minute)

In [34]:

#take out the ids for non districts:
districts_list_brazil = districts_list_brazil[:-3]

ndvi_array = get_ndvi_10_year_fixed(districts_list_brazil)
brazil_string = recent_data_to_string(ndvi_array, "Tocantins Province in Brazil NDVI Anomalies")
print(brazil_string)


Tocantins Province in Brazil NDVI Anomalies 
Wanderlândia - Vegetation Health: 10% above average - Significant high in 19 years 
Palmeirópolis - Vegetation Health: 8% above average - Significant high in 19 years 
Angico - Vegetation Health: 8% above average - Significant high in 19 years 
Luzinópolis - Vegetation Health: 8% above average - Significant high in 19 years 
Cachoeirinha - Vegetation Health: 9% above average - Significant high in 19 years 
Riachinho - Vegetation Health: 9% above average - Significant high in 19 years 
Darcinópolis - Vegetation Health: 10% above average - Significant high in 19 years 
Maurilândia do Tocantins - Vegetation Health: 10% above average - Significant high in 19 years 
Santa Terezinha do Tocantins - Vegetation Health: 13% above average - Significant high in 19 years 
São Bento do Tocantins - Vegetation Health: 9% above average - Significant high in 19 years 
Ananás - Vegetation Health: 11% above average - Significant high in 19 years 
Ponte Alta do 